Higgs Fit

In [1]:
   TTree tree("tree","tree");
   int nevt = tree.ReadFile("Hgg.txt","x");
   if (nevt <= 0) {
      Error("fitHgg","Error reading data from input file ");
   }
   std::cout << "Read " << nevt << " from the file " << std::endl;

Read 30770 from the file 


We make now the RooFit model using the RooWorkspace Factory 

In [2]:
 RooWorkspace w("w");
   w.factory("x[110,160]");  // invariant mass
   
   w.factory("nbackground[10000, 0, 10000]");
   //w.factory("Exponential::z1(x, a1[-1,-10,0])");
   w.var("nbackground")->setVal(nevt);
   w.var("nbackground")->setMin(0.1*nevt);
   w.var("nbackground")->setMax(10*nevt);

   // create exponential model as two components
   w.factory("a1[ 7.5, -500, 500]");
   w.factory("a2[-1.5, -500, 500]");
   w.factory("expr::z('-(a1*x/100 + a2*(x/100)^2)', a1, a2, x)");
   w.factory("Exponential::bmodel(z, 1)");

   // signal model   
   w.factory("nsignal[100, 0.0, 1000.0]");
   //w.factory("mass[%f, %f, %f]' % (massguess, massmin, massmax))
   w.factory("mass[130, 110, 150]");
   w.factory("width[1, 0.5, 5]");
   w.factory("Gaussian::smodel(x, mass, width)");
   RooAbsPdf * smodel = w.pdf("smodel");

   w.factory("SUM::model(nbackground*bmodel, nsignal*smodel)");
   RooAbsPdf * model = w.pdf("model");
   RooRealVar * x = w.var("x");


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



In [3]:
w.Print();


RooWorkspace(w) w contents

variables
---------
(a1,a2,mass,nbackground,nsignal,width,x)

p.d.f.s
-------
RooExponential::bmodel[ x=z c=1 ] = 0.000616625
RooAddPdf::model[ nbackground * bmodel + nsignal * smodel ] = 0.000610556
RooGaussian::smodel[ x=x mean=mass sigma=width ] = 3.72665e-06

functions
--------
RooFormulaVar::z[ actualVars=(a1,a2,x) formula="-(x[0]*x[2]/100+x[1]*(x[2]/100)^2)" ] = -7.39125



We create now the RooFit data set importing the data from the ROOT tree

In [4]:
RooDataSet data("data","data",*w.var("x"),RooFit::Import(tree) );

[#1] INFO:Eval -- RooAbsReal::attachToTree(x) TTree Float_t branch x will be converted to double precision


we now fit the data

In [5]:
RooFitResult * r = nullptr;
RooPlot * plot = nullptr;

In [6]:
r = model->fitTo(data, RooFit::Minimizer("Minuit2"),RooFit::Save(true), RooFit::Offset(true));

input_line_42:2:19: error: reference to 'data' is ambiguous
 r = model->fitTo(data, RooFit::Minimizer("Minuit2"),RooFit::Save(true), RooFit::Offset(true));
                  ^
input_line_39:2:13: note: candidate found by name lookup is '__cling_N55::data'
 RooDataSet data("data","data",*w.var("x"),RooFit::Import(tree) );
            ^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/iterator:1916:22: note: candidate found by name lookup is 'std::__1::data'
constexpr const _Ep* data(initializer_list<_Ep> __il) noexcept { return __il.begin(); }
                     ^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/iterator:1898:6: note: candidate found by name lookup is 'std::__1::data'
auto data(_Cont& __c)
     ^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/iterator:1905:6: note: candidate found by name lookup is 's

Plot of the fit result

In [7]:
plot = w.var("x")->frame();
data.plotOn(plot);
model->plotOn(plot);
model->plotOn(plot, RooFit::Components("bmodel"),RooFit::LineStyle(kDashed));
model->plotOn(plot, RooFit::Components("smodel"),RooFit::LineColor(kRed));
model->paramOn(plot);

plot->Draw();
c1->Draw();

input_line_43:3:1: error: reference to 'data' is ambiguous
data.plotOn(plot);
^
input_line_39:2:13: note: candidate found by name lookup is '__cling_N55::data'
 RooDataSet data("data","data",*w.var("x"),RooFit::Import(tree) );
            ^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/iterator:1916:22: note: candidate found by name lookup is 'std::__1::data'
constexpr const _Ep* data(initializer_list<_Ep> __il) noexcept { return __il.begin(); }
                     ^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/iterator:1898:6: note: candidate found by name lookup is 'std::__1::data'
auto data(_Cont& __c)
     ^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/iterator:1905:6: note: candidate found by name lookup is 'std::__1::data'
auto data(const _Cont& __c)
     ^
/Applications/Xcode.app/Contents/Developer/Too

In [8]:
w.writeToFile("HiggsModel.root",true);